In [6]:
import numpy as np
import numpy
import datetime
import pandas as pd
from datetime import datetime
import vaex as vx

In [7]:
import matplotlib.pyplot as plt
from  plotly.offline import iplot
import plotly as py 
import plotly.express as px

In [8]:
import cufflinks as cf 
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [9]:
def convert_to_datetime(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y %H:%M:%S'))

def convert_to_date(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y'))

def check_wekend(date_string):
    a = datetime.strptime(str(date_string), '%d/%m/%Y')
    weekno = a.weekday()
    if weekno < 5:
        return "Weekday"
    else:  # 5 Sat, 6 Sun
        return "Weekend"
    
def check_month(i):
    if i <= 10:
        return "July-Oct"
    else:
        return "Nov-Dec"
    
def run_preprocess(data):
    data['date_time']  = (data['TICKET DATE'] + ' ' + data['TICKET TIME']).apply(convert_to_datetime)
    data['ticket_date']  = data['TICKET DATE'].apply(convert_to_date)
#     data["Day"] = data["date_time"].dt.day_name
#     data["Hour"] = data["date_time"].dt.hour
#     data["Month"] = data["date_time"].dt.month
#     data['day_type'] = data['TICKET DATE'].apply(check_wekend)
#     data['Period'] = data['Month'].apply(check_month)

In [10]:
full_data = vx.open('full_data.hdf5')

In [12]:
run_preprocess(full_data)

In [13]:
full_data = full_data[full_data['ticket_date'] != '2019-08-15']

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Jatin\.conda\envs\thesis\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Jatin\.conda\envs\thesis\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Jatin\.conda\envs\thesis\lib\multiprocessing\pool.py", line 519, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Users\Jatin\.conda\envs\thesis\lib\multiprocessing\pool.py", line 499, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "C:\Users\Jatin\.conda\envs\thesis\lib\multiprocessing\connection.py", line 879, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
  File "C:\Users\Jatin\.conda\envs\thesis\lib\multiprocessing\connection.py", line 811, in _exhaustive_wait
    res = _winapi.WaitForMultipleObjects(L, False, timeout)
ValueError: need at most 63 handles, got a sequence of length 194


In [15]:
def convert_to_datetime(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d-%m-%Y %H:%M:%S'))

def convert_to_date(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y'))

def check_wekend(date_string):
    a = datetime.strptime(str(date_string), '%d/%m/%Y')
    weekno = a.weekday()
    if weekno < 5:
        return "Weekday"
    else:  # 5 Sat, 6 Sun
        return "Weekend"
    
def check_month(i):
    if i <= 10:
        return "July-Oct"
    else:
        return "Nov-Dec"
    
def run_preprocess(data):
    data['date_time']  = data['date_time'].apply(convert_to_datetime)
#     data['ticket_date']  = data['TICKET DATE'].apply(convert_to_date)
#     data["Day"] = data["date_time"].dt.day_name()
#     data["Hour"] = data["date_time"].dt.hour
#     data["Month"] = data["date_time"].dt.month
#     data['day_type'] = data['TICKET DATE'].apply(check_wekend)
#     data['Period'] = data['Month'].apply(check_month)

months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
dic = {months[i]:i+1 for i in range(12)}
def month_indexer(d):
    temp = d.split('-')[1]
    new_d= d.replace(temp,str(dic[temp]))
    return new_d

def trip_time_finder(t):
    return trips[trips['TRIP_ID'] == t]['TRIP_START_TIME'].values[0]

In [42]:
column_names = ['TRIP_ID','ROUTE_ORIGIN_STOP','ROUTE_DESTINATION_STOP','USER_COUNT']

all_routes = full_data['ROUTENAME'].unique()

trips = full_data[['TRIP_ID','TRIP_START_TIME']]
trips = trips.to_pandas_df()
trips.drop_duplicates(inplace=True)

from tqdm import tqdm

for r in tqdm(all_routes):
    
    gtfs_route = pd.read_csv(f"D:\\MTech\\Dissertation\\Data\\GTFS_with_trip_length\\{r}.csv",index_col='Unnamed: 0')
    stop_order = list(gtfs_route['stop_name'].values)
    
    top_route_data = full_data[full_data['ROUTENAME'] == r]
    
    d = {}
    for c in column_names:
        d[c] = top_route_data[c].values
    top_route_data_pd = pd.DataFrame(d)
    
    occupancy_data = []
    for t in (top_route_data_pd['TRIP_ID'].unique()):
        
        o_dict = {}
        d_dict = {}
        
        single_trip_data = top_route_data_pd[top_route_data_pd['TRIP_ID'] == t]
        o = single_trip_data.groupby(by='ROUTE_ORIGIN_STOP')["USER_COUNT"].sum()
        d = single_trip_data.groupby(by='ROUTE_DESTINATION_STOP')["USER_COUNT"].sum()
        for s in o.index:
            o_dict[s] = o_dict.get(s,0) + o[s]
        for s in d.index:
            d_dict[s] = d_dict.get(s,0) - d[s]
        
        temp = []
        count = 0
        temp.append(t)
        
        for i in stop_order:
            temp.append(count)
            count += o_dict.get(i,0)
            count += d_dict.get(i,0)
            
        occupancy_data.append(temp)
        
    column = ['TRIP_ID'] + stop_order
    occupancy_df = pd.DataFrame(occupancy_data,columns=column)
    
    occupancy_df = pd.merge(left=occupancy_df,right=trips,how='inner')
    
    occupancy_df['date_time'] = occupancy_df['TRIP_START_TIME'].apply(month_indexer)
    occupancy_df.drop(columns='TRIP_START_TIME',inplace=True)
    occupancy_df.to_csv(f'D:\\MTech\\Dissertation\\Data\\data_for_bus_occupancy_prediction\\{r}.csv',index=False)

100%|████████████████████████████████████████████████████████████████████████████| 550/550 [37:11:11<00:00, 243.40s/it]


In [43]:
import os

files = os.listdir('D:\MTech\Dissertation\Data\data_for_bus_occupancy_prediction')
files

['101BDOWN.csv',
 '101BUP.csv',
 '101EXTDOWN.csv',
 '101EXTUP.csv',
 '108DOWN.csv',
 '108STLDOWN.csv',
 '108STLUP.csv',
 '108UP.csv',
 '114+990EDOWN.csv',
 '114+990EUP.csv',
 '114+990UP.csv',
 '114ST+901DOWN.csv',
 '125DOWN.csv',
 '125EXTDOWN.csv',
 '125EXTUP.csv',
 '129ADOWN.csv',
 '129AUP.csv',
 '129DOWN.csv',
 '129UP.csv',
 '143DOWN.csv',
 '143UP.csv',
 '157ADOWN.csv',
 '157AUP.csv',
 '157STLDOWN.csv',
 '161ADOWN.csv',
 '161AUP.csv',
 '161DOWN.csv',
 '161UP.csv',
 '165ADOWN.csv',
 '165AUP.csv',
 '165DOWN.csv',
 '165LINKSTLDOWN.csv',
 '165LINKSTLUP.csv',
 '165UP.csv',
 '181ADOWN.csv',
 '181AUP.csv',
 '181DOWN.csv',
 '181UP.csv',
 '182ACLUP.csv',
 '182ADOWN.csv',
 '182AUP.csv',
 '185DOWN.csv',
 '185LNKSTL2DOWN.csv',
 '185LNKSTLUP.csv',
 '185LSTLDOWN.csv',
 '185LSTLUP.csv',
 '185STLDOWN.csv',
 '185STLUP.csv',
 '185UP.csv',
 '202DOWN.csv',
 '202UP.csv',
 '205DOWN.csv',
 '205LSTLDOWN.csv',
 '205UP.csv',
 '206ADOWN.csv',
 '206AUP.csv',
 '208DOWN.csv',
 '208LNKSTLUP.csv',
 '208UP.csv',
 '2

In [44]:
occupancy_df

,TRIP_ID,Dichau Kalan Depot,Nirmal Vihar,Udaseen Ashram,Dichau Chowk,Health Centre,Najafgarh Delhi Gate,Chawla Stand Najafgarh,BDO Office Roshan Pura,Durga Vihar,...,Jhatikara Crossing,Rewla Taj Pur More,Chhawla School,Chhawla Village Kangan Heri Road,Rishal Vihar,Kangan Heri - I,Daulat Pur Crossing,Shikar Pur Crossing,Daulat Pur Village,date_time
0,T11010010719111301,0,0,0,0,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,01-7-2019 11:13:56
1,T10686020719084301,0,1,1,1,1,1,2,2,1,...,1,0,0,0,0,0,0,0,0,02-7-2019 08:43:14
2,T11952030719061701,0,0,0,0,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,03-7-2019 06:17:51
3,T10193040719063001,0,2,2,2,2,2,2,2,0,...,0,0,0,0,0,0,0,0,0,04-7-2019 06:30:14
4,T11391050719065801,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,05-7-2019 06:58:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,T12656131219072701,0,3,3,3,3,3,3,3,0,...,0,0,0,0,0,0,0,0,0,13-12-2019 07:27:03
90,T12141141219063701,0,0,0,0,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,14-12-2019 06:37:10
91,T10059161219062801,0,0,0,0,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,16-12-2019 06:28:12
92,T12602181219084101,0,0,0,0,0,0,4,4,4,...,4,0,0,0,0,0,0,1,1,18-12-2019 08:41:19


In [45]:
import numpy as np
from datetime import datetime

def find_week(d):
    date = d
    timestamp = (date - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    dt = datetime.utcfromtimestamp(timestamp)
    year, week_num, weekday = dt.isocalendar()
    return week_num

def create_stamp(d):
    return (pd.Timestamp(d).hour) + (pd.Timestamp(d).minute)/100

In [46]:
def check_wekend(date_string):
    a = datetime.strptime(str(date_string), '%d/%m/%Y')
    weekno = a.weekday()
    if weekno < 5:
        return "Weekday"
    else:  # 5 Sat, 6 Sun
        return "Weekend"
    
def check_month(i):
    if i <= 10:
        return "July-Oct"
    else:
        return "Nov-Dec"
    
def run_preprocess(data):
    data["Day"] = data["date_time"].dt.day_name()
    data["Month"] = data["date_time"].dt.month
    data['Period'] = data['Month'].apply(check_month)

In [47]:
def convert_to_datetime(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d-%m-%Y %H:%M:%S'))

for f in tqdm(files):
    df = pd.read_csv(r"D:/MTech/Dissertation/Data/data_for_bus_occupancy_prediction/"+f)
    stops = df.columns[1:-1]
    
    df_list = []
    
    for stop in stops:
        df1 = df[[stop,'date_time']]
        df1['date_time']  = df1['date_time'].apply(convert_to_datetime)
        resample_dict = {stop:'mean'}
        df1 = df1.resample('15T',on='date_time').agg(resample_dict)
        df1.reset_index(inplace=True)
        df1.fillna(0,inplace=True)
        df1['stop_name'] = stop
        df1.rename(columns = {stop:'user_count'},inplace=True)
        df_list.append(df1)

    oneroute_occupancy = pd.concat(df_list)
    oneroute_occupancy['week_number'] = oneroute_occupancy['date_time'].apply(find_week)
    oneroute_occupancy['time_stamp'] = oneroute_occupancy['date_time'].apply(create_stamp)
    run_preprocess(oneroute_occupancy)
    oneroute_occupancy.drop(columns='date_time',inplace=True)
    oneroute_occupancy.to_csv(f'D:\\MTech\\Dissertation\\Data\\data_for_model_input_bus_occupancy_prediction\\{f}',index=False)

  0%|                                                                                          | 0/550 [00:00<?, ?it/s]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  0%|▏                                                                               | 1/550 [00:08<1:15:08,  8.21s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  0%|▎                                                                               | 2/550 [00:15<1:08:45,  7.53s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  1%|▍                                                                               | 3/550 [00:24<1:14:57, 

  5%|████                                                                           | 28/550 [08:15<3:53:31, 26.84s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  5%|████▏                                                                          | 29/550 [08:26<3:11:04, 22.00s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  5%|████▎                                                                          | 30/550 [08:37<2:41:42, 18.66s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

  6%|████▍                                                                          | 31/550 [09:04<3:05:13, 

 10%|████████                                                                       | 56/550 [15:30<2:31:49, 18.44s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 10%|████████▏                                                                      | 57/550 [15:50<2:36:21, 19.03s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 11%|████████▎                                                                      | 58/550 [15:53<1:57:10, 14.29s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 11%|████████▍                                                                      | 59/550 [16:10<2:03:31, 

 15%|████████████                                                                   | 84/550 [24:27<2:51:18, 22.06s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 15%|████████████▏                                                                  | 85/550 [24:30<2:05:39, 16.21s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 16%|████████████▎                                                                  | 86/550 [24:33<1:34:43, 12.25s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 16%|████████████▍                                                                  | 87/550 [24:49<1:44:03, 

 20%|███████████████▉                                                              | 112/550 [29:38<2:25:21, 19.91s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 21%|████████████████                                                              | 113/550 [29:42<1:49:29, 15.03s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 21%|████████████████▏                                                             | 114/550 [30:06<2:09:23, 17.81s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 21%|████████████████▎                                                             | 115/550 [30:17<1:55:04, 

...

 76%|███████████████████████████████████████████████████████████▌                  | 420/550 [1:57:52<25:52, 11.94s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 77%|███████████████████████████████████████████████████████████▋                  | 421/550 [1:57:53<18:45,  8.72s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 77%|███████████████████████████████████████████████████████████▊                  | 422/550 [1:58:05<20:51,  9.78s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 77%|███████████████████████████████████████████████████████████▉                  | 423/550 [1:58:28<28:44, 

 81%|███████████████████████████████████████████████████████████████▌              | 448/550 [2:04:56<37:44, 22.20s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 82%|███████████████████████████████████████████████████████████████▋              | 449/550 [2:05:27<42:06, 25.01s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 82%|███████████████████████████████████████████████████████████████▊              | 450/550 [2:05:58<44:26, 26.66s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 82%|███████████████████████████████████████████████████████████████▉              | 451/550 [2:06:19<41:18, 

 87%|███████████████████████████████████████████████████████████████████▌          | 476/550 [2:13:07<18:56, 15.36s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 87%|███████████████████████████████████████████████████████████████████▋          | 477/550 [2:13:16<16:16, 13.37s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 87%|███████████████████████████████████████████████████████████████████▊          | 478/550 [2:13:32<16:59, 14.16s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 87%|███████████████████████████████████████████████████████████████████▉          | 479/550 [2:14:04<23:06, 

 92%|███████████████████████████████████████████████████████████████████████▍      | 504/550 [2:21:38<14:13, 18.55s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 92%|███████████████████████████████████████████████████████████████████████▌      | 505/550 [2:21:53<13:09, 17.54s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 92%|███████████████████████████████████████████████████████████████████████▊      | 506/550 [2:22:06<11:56, 16.27s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 92%|███████████████████████████████████████████████████████████████████████▉      | 507/550 [2:22:25<12:06, 

 97%|███████████████████████████████████████████████████████████████████████████▍  | 532/550 [2:28:44<05:07, 17.08s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 97%|███████████████████████████████████████████████████████████████████████████▌  | 533/550 [2:29:03<05:01, 17.72s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 97%|███████████████████████████████████████████████████████████████████████████▋  | 534/550 [2:29:25<05:07, 19.20s/it]C:\Users\Jatin\AppData\Local\Temp\3\ipykernel_25196\4185787878.py:6: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

 97%|███████████████████████████████████████████████████████████████████████████▊  | 535/550 [2:29:47<04:57, 